In [1]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

# A$^*$-IDA$^*$ Search

In [2]:
import heapq

The function `search` takes four arguments to solve a *search problem*:
- `start` is the start state of the search problem,
- `goal` is the goal state, and
- `next_states` is a function with signature $\texttt{next_states}:Q \rightarrow 2^Q$, where $Q$ is the set of states.
  For every state $s \in Q$, $\texttt{next_states}(s)$ is the set of states that can be reached from $s$ in one step.
- `heuristic` is a function that takes two states as arguments.  It 
  returns an estimate of the length of the shortest path between these
  states.
- `size` is the maximum number states that A$^*$ search is allowed 
   to explore.
   
If successful, `search` returns a path from `start` to `goal` that is a solution of the search problem
$$ \langle Q, \texttt{next_states}, \texttt{start}, \texttt{goal} \rangle. $$

The function `search` implements A$^*$-IDA$^*$ search.
The main idea of A$^*$-IDA$^*$ is to run an $\mathrm{A}^*$ search from the node 
`start` until memory is more or less exhausted.  Then, we start $\mathrm{IDA}^*$ from the node `goal` node and search towards the node 
`start` until we find any of the nodes discovered by the $\mathrm{A}^*$ search 
that had been started from the node `start` node.

In [3]:
def search(start, goal, next_states, heuristic, size):
    Parent    = { start: start }
    PrioQueue = [ (heuristic(start, goal), [start]) ]
    while len(Parent) < size and PrioQueue:
        _, Path = heapq.heappop(PrioQueue)
        state   = Path[-1]
        if state == goal:
            return Path
        for ns in next_states(state):
            if ns not in Parent:
                prio = heuristic(ns, goal) + len(Path)
                Parent[ns] = state
                heapq.heappush(PrioQueue, (prio, Path + [ns]))
    Path = id_search(goal, start, next_states, heuristic, Parent)
    return path_to(Path[-1], Parent) + Path[::-1][1:]

In [4]:
def id_search(goal, start, next_states, heuristic, Parent):
    limit = heuristic(start, goal)
    while True:
        print(f'limit = {limit}')
        Path = dl_search([goal], start, next_states, limit, heuristic, Parent)
        if isinstance(Path, list):
            return Path
        limit = Path

In [5]:
def dl_search(Path, start, next_states, limit, heuristic, Parent):
    state = Path[-1]
    total = len(Path) - 1 + heuristic(state, start)
    if total > limit:
        return total
    if state in Parent:
        return Path
    smallest = float('Inf')
    for ns in next_states(state):
        if ns not in Path:
            result = dl_search( Path + [ns], start, next_states, 
                                limit, heuristic, Parent
                              )
            if isinstance(result, list):
                return result
            smallest = min(smallest, result)
    return smallest

Given a `state` and a parent dictionary `Parent`, the function `path_to` returns a path leading to the given `state`.

In [6]:
def path_to(state, Parent):
    p = Parent[state]
    if p == state:
        return [state]
    return path_to(p, Parent) + [state]

Let's draw the start state and animate the solution that has been found.

In [7]:
%run Sliding-Puzzle.ipynb

In [8]:
%load_ext memory_profiler

In [9]:
%%time
%memit Path = search(start, goal, next_states, manhattan, 1000)
print(len(Path)-1)

limit = 21
limit = 23
limit = 25
limit = 27
limit = 29
peak memory: 100.83 MiB, increment: 0.80 MiB
33
CPU times: user 75.7 ms, sys: 13.7 ms, total: 89.4 ms
Wall time: 638 ms


In [ ]:
animation(Path)

In [10]:
%%time
Path = search(start2, goal2, next_states, manhattan, 10000)
print(len(Path)-1)

limit = 20
limit = 22
limit = 24
36
CPU times: user 179 ms, sys: 7.68 ms, total: 186 ms
Wall time: 192 ms


In [ ]:
animation(Path)